## 1.2 Экспортируем из OSM


В ооооочень большом числе проектов используются данные OpenStreetMap. Есть множество способов, как их оттуда экспортировать.

Так как мы собираемся анализировать пространственные данные на Python, то и необходимые данные мы можем получить, не закрывая Jupyter Notebook. Для этого воспользуемся библиотекой OSMnx


Для начала стоит определить территорию, на которую мы хотим получить данные. Пусть это будет Центральный район Санкт-Петербурга. Воспользуемся функцией geocode_to_gdf, чтобы получить границы района.


In [ ]:
area = ox.geocode_to_gdf('Центральный район, Санкт-Петербург')

Посмотрим, что мы получили (а еще в методе explore зададим параметр tiles и поменяем базовую карту OSM на Positron)


In [ ]:
area.explore(tiles='cartodbpositron')

Невероятно! Но это именно то, что нам нужно – границы Центрального района Санкт-Петербурга


Теперь попробуем выгрузить здания для Центрального района. Для этого создадим переменную tags, куда передадим ключ нужных нам объектов - buildings.
В функцию geometries_from_place передадим название района и теги объектов

Подробнее о ключах и значениях данных OSM можно прочитать <a href="https://wiki.openstreetmap.org/wiki/Map_features" target="_blank">тут</a>


In [ ]:
tags = {'building': True}   

buildings = ox.features_from_place('Центральный район, Санкт-Петербург', tags)  #in the OSMnx versions before 1.7.0 use ox.geometries.geometries_from_place())

buildings.explore(tiles='cartodbpositron',tooltip=None)

Мы получили здания!

Теперь выберем только полигональные (и мультиполигональные) объекты


In [ ]:
buildings = buildings.loc[buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

И смело можем сохранять!

Спойлер – при сохранении возникнет ошибка, так как некоторые поля содержат данные в формате списка (list). Это не подходит для сохранения, поэтому мы это исправим: перезапишем поля как строки(string)


In [ ]:
for col in buildings.columns:
    if any(isinstance(val, list) for val in buildings[col]):
        buildings = buildings.astype({col: str})

И теперь точно можем сохранять!


In [ ]:
#buildings.to_file('data/spb_central_build.geojson')